In [1]:
import os
from re import S
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
import librosa
import yaml

from model import RawNet



In [4]:
emb = torch.load('/dataa/phucdt/bio/distillation/wav2vec_bio_emb/LA_E_1000147')


In [5]:
print(len(emb))

192


In [2]:
config_path = "model_config_RawNet.yaml"
model_path = "models/model_LA_CCE_100_64_0.0001/epoch_71.pth"
device = 'cpu' 
with open(config_path, 'r') as f_yaml:
    parser1 = yaml.safe_load(f_yaml)

model = RawNet(parser1['model'], device).to(device)
model.load_state_dict(torch.load(model_path,map_location=device))

<All keys matched successfully>

In [3]:
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def parse_input(file_path):
    cut = 64600  # take ~4 sec audio (64600 samples)
    X, fs = librosa.load(file_path, sr=16000)
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    return x_inp.unsqueeze(0).to(device)

In [4]:
wav1 = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
#wav2 = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_5464494.flac"
wav3 = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_2925019.flac"
wav4 = "/root/dataset/Speech/AIHub_Studio/wavs/01FHSH0028/01FHSH0028_00122.wav"
wav5 = "/root/biological/asvspoof2021/LA/Baseline-RawNet2-bio/Demo/prof_16k.flac"

In [7]:
model.eval()
# Load the audio file that we need to check whether it is fake voice or not.
wav1 = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
#wav2 = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_5464494.flac"
#wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_2925019.flac"
#wav = "/root/dataset/Speech/AIHub_Studio/wavs/01FHSH0028/01FHSH0028_00122.wav"
wav = "/dataa/phucdt/bio/test_data/LA_E_1665632.flac" # bonafide
wav = "/dataa/phucdt/bio/test_data/LA_E_5085671.flac" # spoof

# Extract the feature of the audio

x_inp = parse_input(wav)
print(x_inp)
# use the model to calculate the prediction value
# out, _ = model(x_inp)
out, _ = model(x_inp)

# out = [0.123345135,-0.1234145124]

# Convert into readable format
_, pred = out.max(dim=1)
if pred:
    print("bonafide") #real voice
else:
    print("spoof") #fake voice

tensor([[ 0.0004,  0.0003,  0.0004,  ..., -0.0035,  0.0002, -0.0019]])
spoof


In [10]:
wav = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
x_inp = parse_input(wav)
out, _ = model(x_inp)
print(pred, _)

tensor([0]) tensor([[ 0.0198, -0.0488, -0.0493,  ..., -0.0502, -0.6838,  0.9603]],
       grad_fn=<AddmmBackward>)


In [11]:
model.eval()
# Load the audio file that we need to check whether it is fake voice or not.
wav = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
x_inp = parse_input(wav)
def printsave(*a):
    file = open('output.txt','a')
    print(*a)
    print(*a,file=file)
printsave(x_inp)

tensor([[ 0.0004,  0.0003,  0.0004,  ..., -0.0035,  0.0002, -0.0019]])


# Android Conversion

In [1]:
import os
from re import S
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
import librosa
import yaml

from model import RawNet

/home/phucdt/anaconda3/envs/bio/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_path = "model_config_RawNet.yaml"
model_path = "models/model_LA_CCE_100_64_0.0001/epoch_71.pth"
device = 'cpu' 
with open(config_path, 'r') as f_yaml:
    parser1 = yaml.safe_load(f_yaml)

model = RawNet(parser1['model'], device).to(device)
model.load_state_dict(torch.load(model_path,map_location=device))

<All keys matched successfully>

In [3]:
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def parse_input(file_path):
    cut = 64600  # take ~4 sec audio (64600 samples)
    X, fs = librosa.load(file_path, sr=16000)
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    return x_inp.unsqueeze(0).to(device)

In [4]:
f = open("data.txt", 'r')
data = []
while True:
    line = f.readline().strip()
    if not line: break
    data.append(line)
    #print(line)
f.close()
print("데이터 개수 :",len(data))

데이터 개수 : 71237


In [6]:
model.eval()
import torch
from torch import Tensor
from torch.utils.mobile_optimizer import optimize_for_mobile

class BTSDetect(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, wavforms: Tensor):
        logits, _ = self.model(wavforms)
        _, pred = logits.max(dim=1)
        if pred:
            return 0 # bonafide
        else:
            return 1 # fake
        
# Load the audio file that we need to check whether it is fake voice or not.
wav = "/dataa/phucdt/bio/test_data/LA_E_1665632.flac" # bonafide
wav = "/dataa/phucdt/bio/test_data/LA_E_5085671.flac" # spoof

_model = BTSDetect(model)
# # Apply quantization / script / optimize for motbile

quantized_model = torch.quantization.quantize_dynamic(
    _model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
scripted_model = torch.jit.script(quantized_model)
optimized_model = optimize_for_mobile(scripted_model)


# Sanity check

# print('Result:', optimized_model(x_inp))

In [ ]:
model.eval()
# Load the audio file that we need to check whether it is fake voice or not.
wav = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac" #spoof
#wav = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_1665632.flac" #bonafides

x_inp = parse_input(wav)
print(x_inp)
# use the model to calculate the prediction value
# out, _ = model(x_inp)
out, _ = model(x_inp)
# Convert into readable format
_, pred = out.max(dim=1)
if pred:
    print("bonafide") #real voice
else:
    print("spoof") #fake voice

In [3]:
import torch
from torch import Tensor
from torch.utils.mobile_optimizer import optimize_for_mobile
import typing

In [4]:
class BTSDetect(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def pad(self, x, max_len: int = 64600):
        x_len = x.shape[0]

        if (x_len>=max_len):
            return x[:max_len]
        else:
            num_repeats = int(max_len/x_len)+1
            return x.tile((1, num_repeats))[:, :max_len][0]
    
    def forward(self, wavforms):
        wav_padded = self.pad(wavforms)
        logits, _ = self.model(wav_padded)
        _, pred = logits.max(dim=1)
        if pred:
            return 0 # bonafide
        else:
            return 1 # fake

_model = BTSDetect(model)


In [5]:
# # Apply quantization / script / optimize for motbile
_model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    _model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
scripted_model = torch.jit.script(quantized_model)
optimized_model = optimize_for_mobile(scripted_model)


# Sanity check
file_path = "/dataa/phucdt/bio/test_data/LA_E_1665632.flac"
data, y = librosa.load(file_path, sr=16000)
print('Result:', optimized_model(Tensor(data)))
optimized_model._save_for_lite_interpreter("btsdetect.ptl")

: 

: 

In [24]:
f = open("data.txt", 'r')
data = []
while True:
    line = f.readline().strip()
    if not line: break
    data.append(line)
    #print(line)
f.close()
print("데이터 개수 :",len(data))

데이터 개수 : 71237


# TEST

In [ ]:
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def parse_input(file_path):
    cut = 64600  # take ~4 sec audio (64600 samples)
    X, fs = librosa.load(file_path, sr=16000)
    print(X,type(X), "here")
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    print(x_inp,type(x_inp), "here2")
    return x_inp.unsqueeze(0).to(device)

In [7]:
model.eval()
# Load the audio file that we need to check whether it is fake voice or not.
wav = "/root/dataset/Dataset/ASVspoof/LA/ASVspoof2019_LA_eval/flac/LA_E_5085671.flac"
# wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_5464494.flac"
# wav = "/root/dataset/ASVspoof/ASVspoof2021_LA_eval/flac/LA_E_2925019.flac"

# wav = "/root/dataset/Speech/AIHub_Studio/wavs/01FHSH0028/01FHSH0028_00122.wav"

# wav = "/root/biological/asvspoof2021/LA/Baseline-RawNet2-bio/Demo/prof_16k.flac"

# Extract the feature of the audio

x_inp = parse_input(wav)
print(x_inp)
# use the model to calculate the prediction value
# out, _ = model(x_inp)
out, _ = model(x_inp)
print("out = ",out, type(out))
# out = [0.123345135,-0.1234145124]

# Convert into readable format
_, pred = out.max(dim=1)
print("pred = ",pred)
if pred:
    print("bonafide") #real voice
else:
    print("spoof") #fake voice

tensor([[ 0.0004,  0.0003,  0.0004,  ..., -0.0035,  0.0002, -0.0019]])
out =  tensor([[-2.1815e-05, -1.0732e+01]], grad_fn=<LogSoftmaxBackward>) <class 'torch.Tensor'>
spoof
